In [1]:
import os
corpora_path = '../data/corpora/CSTNews 6.0'
corpora_dirs = os.listdir(corpora_path)
corpora_dirs = [d for d in corpora_dirs if d not in '.DS_Store']
corpora_dirs.remove('For all the clusters')

In [2]:
news = {}
for news_name in corpora_dirs:
    noticias = os.listdir(corpora_path + '/' + news_name + '/Textos-fonte')
    news[news_name] = {}
    for file_title in noticias:
        if '_titulo' not in file_title:
            news[news_name][file_title] = {'Texto' : [], 'Segmentação' : [], 'Tópico (Rotule apenas Inicio)' : []}
            file = corpora_path + '/' + news_name + '/Textos-fonte/' + file_title
            with open(file, 'r', encoding='utf-8') as text_file:
                lines = text_file.readlines()
                for line in lines:
                    if line != '\n':
                        news[news_name][file_title]['Texto'].append(line)
                        news[news_name][file_title]['Segmentação'].append('')
                        news[news_name][file_title]['Tópico (Rotule apenas Inicio)'].append('')

In [3]:
#news[nome da noticia][noticia da fonte][texto]
news['C22_Cotidiano_DeslizamentoCongonhas']['D5_C22_GPovo_24-07-2007_08h51.txt']['Texto']

['Quase metade dos vôos previstos para decolar na manhã desta terça-feira (24) no Aeroporto de Congonhas, na Zona Sul de São Paulo, foi cancelada, de acordo com informações da Infraero. Das 39 decolagens previstas entre as 6h e as 8h, 19 sofreram cancelamentos ou o equivalente a 48,7% do total. \n',
 'A TAM anunciou o cancelamento de 68 vôos nesta terça (24) e o remanejamento de outros 22. Dos vôos cancelados, 8 passariam pelo Paraná. Dois vôos que sairiam de Curitiba para Congonhas foram remanejados para Guarulhos.\n',
 'A pista auxilar de Congonhas abriu às 6h, apenas para decolagens. A pista principal está fechada há uma semana, desde o acidente com o vôo 3054 da TAM. \n',
 'Oito pousos que deveriam ocorrer entre as 6h e as 8h foram alternados para o Aeroporto Internacional de Cumbica, em Guarulhos, zona metropolitana de São Paulo.\n',
 'Cumbica opera por instrumentos desde as 7h15, por causa da visibilidade ruim. Das 48 partidas previstas entre a meia-noite e as 8h, quatro partiram

In [16]:
import pandas as pd

for new in news:
    #nome da planilha
    name = '../topicos_xls/' + new[0:3] + '.xlsx'
    #criando um objeto writer do excel
    writer = pd.ExcelWriter(name, engine='xlsxwriter')
    #contador de numero de paginas da planilha
    num = 1
    for newspaper in news[new]:
        #criando um dataframe para a noticia no dicionario
        df = pd.DataFrame(news[new][newspaper], columns=['Texto','Segmentação','Tópico (Rotule apenas Inicio)'])
        #nome da pagina da planilha
        sname = (new[0:3] + '.' + str(num)).replace('_', '')
        #criando a nova pagina na planilha
        df.to_excel(writer, index=False, sheet_name=sname)
        
        #inicializando o workbook e setando qual pagina da planilha sera trabalhada
        workbook  = writer.book
        worksheet = writer.sheets[sname]        
        
        #formatacao para as colunas
        formatA = workbook.add_format({'text_wrap': True,
                                      'font_color': '#000000',
                                      'font_size' : 10,
                                      'bold': False,
                                      'valign': 'justify'})

        #formatacao da coluna (coluna, largura, tipo fortacao)
        worksheet.set_column('A:A', 70, formatA)
        worksheet.set_column('B:B', 15, formatA)
        worksheet.set_column('C:C', 30, formatA)        
        
        #criando uma formatacao para a linha de cabecalho
        header_format = workbook.add_format({
            'bold': True,
            'text_wrap': True,
            'valign': 'center',
            'valign': 'vcenter',
            'fg_color': '#1E90FF',
            'font_color': '#F8F8FF',
            'border': 1})
        
        #escrevendo o cabeçalho na primeira linha de cada caluna
        for col_num, value in enumerate(df.columns.values):
            worksheet.write(0, col_num, value, header_format)
        
        num = num+1
        
    writer.save()